In [90]:
import gEconpy as ge
import gEconpy.plotting as gp
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import statsmodels.api as sm

config = {
    'figure.figsize':(14,4),
    'figure.dpi':144,
    'figure.facecolor':'white',
    'axes.grid':True,
    'grid.linestyle':'--',
    'grid.linewidth':0.5,
    'axes.spines.top':False,
    'axes.spines.bottom':False,
    'axes.spines.left':False,
    'axes.spines.right':False
}

plt.rcParams.update(config)

In [91]:
def make_var_names(var, n_lags, reg):
    names = [f'L1.{var}']
    for lag in range(1, n_lags + 1):
        names.append(f'D{lag}L1.{var}')
    if reg != 'n':
        names.append('Constant')
    if 't' in reg:
        names.append('Trend')

    return names


def ADF_test_summary(df, maxlag=None, autolag='BIC', missing='error'):
    if missing == 'error':
        if df.isna().any().any():
            raise ValueError("df has missing data; handle it or pass missing='drop' to automatically drop it.")
            
    if isinstance(df, pd.Series):
        df = df.to_frame()
        
    for series in df.columns:
        data = df[series].copy()
        if missing == 'drop':
            data.dropna(inplace=True)
            
        print(series.center(110))
        print(('=' * 110))
        line = 'Specification' + ' ' * 15 + 'Coeff' + ' ' * 10 + 'Statistic' + ' ' * 5 + 'P-value' + ' ' * 6 + 'Lags' + ' ' * 6 + '1%'
        line += ' ' * 10 + '5%' + ' ' * 8 + '10%'
        print(line)
        print(('-' * 110))
        spec_fixed = False
        for i, (name, reg) in enumerate(zip(['Constant and Trend', 'Constant Only', 'No Constant'], ['ct', 'c', 'n'])):
            stat, p, crit, regresult = sm.tsa.adfuller(data, regression=reg, regresults=True, maxlag=maxlag,
                                                       autolag=autolag)
            n_lag = regresult.usedlag
            gamma = regresult.resols.params[0]
            names = make_var_names(series, n_lag, reg)
            reg_coefs = pd.Series(regresult.resols.params, index=names)
            reg_tstat = pd.Series(regresult.resols.tvalues, index=names)
            reg_pvals = pd.Series(regresult.resols.pvalues, index=names)

            line = f'{name:<21}{gamma:13.3f}{stat:15.3f}{p:13.3f}{n_lag:11}{crit["1%"]:10.3f}{crit["5%"]:12.3f}{crit["10%"]:11.3f}'
            print(line)

            for coef in reg_coefs.index:
                if coef in name:
                    line = f"\t{coef:<13}{reg_coefs[coef]:13.3f}{reg_tstat[coef]:15.3f}{reg_pvals[coef]:13.3f}"
                    print(line)
                    
def plot_sm_results(res, extra_data=None, filter_output='predicted', var_names=None):
    fig = plt.figure(figsize=(14,8))
    
    endog_vars = res.data.ynames
    states = res.states.predicted.columns
    if var_names:
        states = [x for x in states if x in var_names]
    
    gs, plot_locs = gp.prepare_gridspec_figure(n_cols=3, n_plots=len(states))
    
    for i, (name, loc) in enumerate(zip(states, plot_locs)):
        axis = fig.add_subplot(gs[loc])

        mu = getattr(res.states, filter_output)[name]
        sigma = getattr(res.states, filter_output + '_cov').loc[name, name]

        upper = mu + 1.98 * np.sqrt(sigma + 1e-8)
        lower = mu - 1.98 * np.sqrt(sigma + 1e-8)

        start_idx = 1 if filter_output == 'predicted' else 0
        axis.plot(res.data.dates, mu.values[start_idx:], label='Predicted')
        axis.fill_between(res.data.dates, lower.values[start_idx:], upper.values[start_idx:], color='tab:blue', alpha=0.25)

        if name in endog_vars:
            res.data.orig_endog[name].plot(label='Data', ax=axis)
        
        elif extra_data is not None:
            if name in extra_data.columns:
                extra_data[name].plot(label='Data', ax=axis)

        axis.set(title=name)
    fig.tight_layout()
    title_text = 'One-Step Ahead' if filter_output =='predicted' else filter_output.title()
    fig.suptitle(f'Kalman {title_text} Predictions', y=1.05)
    fig.axes[1].legend(bbox_to_anchor=(0.5, 0.98), loc='lower center', bbox_transform=fig.transFigure, ncols=2)

    plt.show()

In [92]:
ukrstat_indicators = [("https://www.ukrstat.gov.ua/operativ/operativ2022/gdn/Vrch/vrch_ed2022_ue.xlsx",2022), #Working hours(monthly,2022)
                      ("https://www.ukrstat.gov.ua/operativ/operativ2021/gdn/Vrch_ed/vrch_ed2021_ue.xlsx",2021), #Working hours(monthly,2021)
                      ("https://www.ukrstat.gov.ua/operativ/operativ2020/gdn/Vrch_ed/vrch_ed2020_ue.xlsx",2020), #Working hours(monthly,2020)
                      ("https://www.ukrstat.gov.ua/operativ/operativ2019/gdn/Vrch_ed/vrch_ed2019_u.xlsx",2019), #Working hours(monthly,2019)
                      ("https://www.ukrstat.gov.ua/operativ/operativ2018/gdn/Vrch_ed/vrch_ed2018_u.xlsx",2018), #Working hours(monthly,2018)
                      ("https://www.ukrstat.gov.ua/operativ/operativ2017/gdn/Vrch_ed/vrch_ed_u.zip",2017), #Working hours(monthly,2017)
                      ("https://www.ukrstat.gov.ua/operativ/operativ2022/gdn/snzp/snzp_ek/smzp_ek_23_ue.xlsx",2023), #Average wages(quarterly,2023)
                      ("https://www.ukrstat.gov.ua/operativ/operativ2022/gdn/snzp/snzp_ek/smzp_ek_22_ue.xlsx",2022), #Average wages(quarterly,2022)
                      ("https://www.ukrstat.gov.ua/operativ/operativ2021/gdn/snzp/snzp_ek/smzp_ek_21_ue.xlsx",2021), #Average wages(quarterly,2021)
                      ("https://www.ukrstat.gov.ua/operativ/operativ2020/gdn/snzp/snzp_ek/smzp_ek_20_ue.xlsx",2020), #Average wages(quarterly,2020)
                      ("https://www.ukrstat.gov.ua/operativ/operativ2019/gdn/snzp/snzp_ek/smzp_ek_u_19.xlsx",2019), #Average wages(quarterly,2019)
                      ("https://www.ukrstat.gov.ua/operativ/operativ2018/gdn/snzp/snzp_ek/smzp_ek_u_18.xlsx",2018), #Average wages(quarterly,2018)
                      ("https://www.ukrstat.gov.ua/operativ/operativ2017/gdn/snzp/snzp_ek/smzp_ek_u.zip",2017), #Average wages(quarterly,2017)
                     ]

In [93]:
def make_year_rename_dict(year):
  return {
      'Січень':f'{year}-01-01',
      'Лютий':f'{year}-02-01',
      'Березень':f'{year}-03-01',
      'Квітень':f'{year}-04-01',
      'Травень':f'{year}-05-01',
      'Червень':f'{year}-06-01',
      'Липень':f'{year}-07-01',
      'Серпень':f'{year}-08-01',
      'Вересень':f'{year}-09-01',
      'Жовтень':f'{year}-10-01',
      'Листопад':f'{year}-11-01',
      'Грудень':f'{year}-12-01'
  }

bilingual_to_ukr_only = {
     'Січень/January':'Січень',
     'Лютий/February':'Лютий',
     'Березень/March':'Березень',
     'Квітень/April':'Квітень',
     'Травень/May':'Травень',
     'Червень/June':'Червень',
     'Липень/July':'Липень',
     'Серпень/August':'Серпень',
     'Вересень/September':'Вересень',
     'Жовтень/October':'Жовтень',
     'Листопад/November':'Листопад',
     'Грудень/December':'Грудень'
}

In [94]:
def load_and_preprocess_ukrstat_labor_data(url, year):
  df = pd.read_excel(url, skiprows=2, header=[0, 1], index_col=[0], skipfooter=2)

  # I found that before 2019 the data is only in Ukranian (no english), so just
  # convert everything to Ukranian only.
  df = df.rename(columns={'год/  hours':'год'})
  df = df.rename(columns=bilingual_to_ukr_only)
  df = df.xs(axis=1, level=1, key='год')
  df = df.rename(columns=make_year_rename_dict(year))
  df.columns = pd.to_datetime(df.columns)

  return df

In [95]:
for url in ukrstat_indicators:
    print (url)

('https://www.ukrstat.gov.ua/operativ/operativ2022/gdn/Vrch/vrch_ed2022_ue.xlsx', 2022)
('https://www.ukrstat.gov.ua/operativ/operativ2021/gdn/Vrch_ed/vrch_ed2021_ue.xlsx', 2021)
('https://www.ukrstat.gov.ua/operativ/operativ2020/gdn/Vrch_ed/vrch_ed2020_ue.xlsx', 2020)
('https://www.ukrstat.gov.ua/operativ/operativ2019/gdn/Vrch_ed/vrch_ed2019_u.xlsx', 2019)
('https://www.ukrstat.gov.ua/operativ/operativ2018/gdn/Vrch_ed/vrch_ed2018_u.xlsx', 2018)
('https://www.ukrstat.gov.ua/operativ/operativ2017/gdn/Vrch_ed/vrch_ed_u.zip', 2017)
('https://www.ukrstat.gov.ua/operativ/operativ2022/gdn/snzp/snzp_ek/smzp_ek_23_ue.xlsx', 2023)
('https://www.ukrstat.gov.ua/operativ/operativ2022/gdn/snzp/snzp_ek/smzp_ek_22_ue.xlsx', 2022)
('https://www.ukrstat.gov.ua/operativ/operativ2021/gdn/snzp/snzp_ek/smzp_ek_21_ue.xlsx', 2021)
('https://www.ukrstat.gov.ua/operativ/operativ2020/gdn/snzp/snzp_ek/smzp_ek_20_ue.xlsx', 2020)
('https://www.ukrstat.gov.ua/operativ/operativ2019/gdn/snzp/snzp_ek/smzp_ek_u_19.xlsx

In [96]:
import re

df_list = []
for url, year in ukrstat_indicators:
  if url.endswith('xlsx') and 'vrch' in url: # This code only works on the excel sheets for hourly wages
    # This is a regex pattern to pull the year out of a string.
    print(url, year)
    df_list.append(load_and_preprocess_ukrstat_labor_data(url, year))

https://www.ukrstat.gov.ua/operativ/operativ2022/gdn/Vrch/vrch_ed2022_ue.xlsx 2022
https://www.ukrstat.gov.ua/operativ/operativ2021/gdn/Vrch_ed/vrch_ed2021_ue.xlsx 2021
https://www.ukrstat.gov.ua/operativ/operativ2020/gdn/Vrch_ed/vrch_ed2020_ue.xlsx 2020
https://www.ukrstat.gov.ua/operativ/operativ2019/gdn/Vrch_ed/vrch_ed2019_u.xlsx 2019
https://www.ukrstat.gov.ua/operativ/operativ2018/gdn/Vrch_ed/vrch_ed2018_u.xlsx 2018


In [97]:
df = pd.concat(df_list, axis=1).T.sort_index()
df = df.iloc[:, :1]
df.rename(columns={'У середньому по економіці': 'N'}, inplace=True)
df.index.name='date'
df

,N
date,
2018-01-01,NaN
2018-02-01,NaN
2018-03-01,NaN
2018-04-01,NaN
2018-05-01,NaN
2018-06-01,NaN
2018-07-01,NaN
2018-08-01,NaN
2018-09-01,NaN


In [98]:
N_monthly = df['N'].copy()
N_quarterly = N_monthly.dropna().resample('QS').sum()
df = df.drop(columns=['N']).join(N_quarterly,how='outer')
df

,N
date,
2018-01-01,NaN
2018-02-01,NaN
2018-03-01,NaN
2018-04-01,NaN
2018-05-01,NaN
2018-06-01,NaN
2018-07-01,NaN
2018-08-01,NaN
2018-09-01,NaN


In [99]:
df.to_csv('Working hours.csv') 

In [100]:
def make_year_rename_dict(year):
  return {
      'І квартал':f'{year}-01-01',
      'ІІ квартал':f'{year}-04-01',
      'ІІІ квартал':f'{year}-07-01',
      'ІV квартал':f'{year}-10-01',
  }

bilingual_to_ukr_only = {
     'І квартал/I quarter':'І квартал',
     'ІІ квартал/II quarter':'ІІ квартал',
     'ІІІ квартал/III quarter':'ІІІ квартал',
     'ІV квартал/IV quarter':'ІV квартал',
     'грн/UAH':'грн',
  }

In [101]:
def load_and_preprocess_ukrstat_wage_data(url, year):
    df = pd.read_excel(url, skiprows=4 if int(year)>2019 else 3, header=[0, 1], index_col=[0], skipfooter=4)
    df=df.rename(columns=bilingual_to_ukr_only)
    df = df.xs(axis=1, level=1, key='грн')
    df=df.rename(columns=make_year_rename_dict(year))

    return df

In [102]:
import re

df_list = []
for url, year in ukrstat_indicators:
  if url.endswith('xlsx') and 'snzp' in url: # This code only works on the excel sheets for average wages
    print(url, year)
    df_list.append(load_and_preprocess_ukrstat_wage_data(url, year))

https://www.ukrstat.gov.ua/operativ/operativ2022/gdn/snzp/snzp_ek/smzp_ek_23_ue.xlsx 2023
https://www.ukrstat.gov.ua/operativ/operativ2022/gdn/snzp/snzp_ek/smzp_ek_22_ue.xlsx 2022
https://www.ukrstat.gov.ua/operativ/operativ2021/gdn/snzp/snzp_ek/smzp_ek_21_ue.xlsx 2021
https://www.ukrstat.gov.ua/operativ/operativ2020/gdn/snzp/snzp_ek/smzp_ek_20_ue.xlsx 2020
https://www.ukrstat.gov.ua/operativ/operativ2019/gdn/snzp/snzp_ek/smzp_ek_u_19.xlsx 2019
https://www.ukrstat.gov.ua/operativ/operativ2018/gdn/snzp/snzp_ek/smzp_ek_u_18.xlsx 2018


In [103]:
df = pd.concat(df_list, axis=1).T.sort_index()
df = df.iloc[:, :1]
df.rename(columns={'У середньому по економіці': 'w'}, inplace=True)
df.index.name='date'
df.index=pd.to_datetime(df.index)
df

,w
date,
2018-01-01,NaN
2018-04-01,NaN
2018-07-01,NaN
2018-10-01,NaN
2019-01-01,9628.960000
2019-04-01,10429.620000
2019-07-01,10731.910000
2019-10-01,11219.590000
2020-01-01,11005.980000


In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 24 entries, 2018-01-01 to 2022-10-01
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   w       19 non-null     float64
dtypes: float64(1)
memory usage: 384.0 bytes


In [104]:
w_monthly = df['w'].copy()
w_quarterly = w_monthly.dropna().resample('QS').sum()
df = df.drop(columns=['w']).join(w_quarterly,how='outer')
df

,w
date,
2018-01-01,NaN
2018-04-01,NaN
2018-07-01,NaN
2018-10-01,NaN
2019-01-01,9628.960000
2019-04-01,10429.620000
2019-07-01,10731.910000
2019-10-01,11219.590000
2020-01-01,11005.980000


In [105]:
df.to_csv('Average wages.csv') 